In [ ]:
!pip install lxml
!pip install spacy

In [ ]:
from lxml import etree
import spacy
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Commedia_Petrocchi_XML_TEI.xml'

In [ ]:
parser = etree.XMLParser(recover=True)
xml_data = etree.parse(file_path, parser)

In [ ]:
cantica = xml_data.xpath('//div1/@id')[0]
tot_canti = xml_data.xpath('//div1[@id="Inferno"]//div2')

In [ ]:
canti = {}

for c in tot_canti:
    h_canto = c.xpath('.//head//text()')
    terzine = []

    for lg in c.xpath('.//lg'):

        terzina = " ".join([l.text.strip() for l in lg.xpath('./l') if l.text.strip()])
        terzine.append(terzina)

    canti[h_canto[0]] = terzine


data = []
for titolo, ter in canti.items():
    i = 1
    for t in ter:
        if len(t.split()) >= 3:
            range_versi = f"{i}_{i+2}"
            data.append([cantica, titolo, range_versi, t])
            i += 3
        else:
            range_versi = f"{i}"
            data.append([cantica, titolo, range_versi, t])
            i += 1

df = pd.DataFrame(data, columns=["cantica", "canto", "range_versi", "terzina"])
df.to_csv("inferno_terzine.csv", index=False, sep=';', encoding='utf-8-sig')
print(df)

      cantica  canto range_versi  \
0     Inferno      I         1_3   
1     Inferno      I         4_6   
2     Inferno      I         7_9   
3     Inferno      I       10_12   
4     Inferno      I       13_15   
...       ...    ...         ...   
1591  Inferno  XXXIV     127_129   
1592  Inferno  XXXIV     130_132   
1593  Inferno  XXXIV     133_135   
1594  Inferno  XXXIV     136_138   
1595  Inferno  XXXIV     139_141   

                                                terzina  
0     Nel mezzo del cammin di nostra vita mi ritrova...  
1     Ahi quanto a dir qual era è cosa dura esta sel...  
2     Tant'è amara che poco è più morte; ma per trat...  
3     Io non so ben ridir com'i' v'intrai, tant'era ...  
4     Ma poi ch'i' fui al piè d'un colle giunto, là ...  
...                                                 ...  
1591  Luogo è là giù da Belzebù remoto tanto quanto ...  
1592  d'un ruscelletto che quivi discende per la buc...  
1593  Lo duca e io per quel cammino ascoso in

In [ ]:
canti = {}

# Processa i canti e crea un dizionario con le terzine come stringhe
for c in tot_canti:
    h_canto = c.xpath('.//head//text()')  # Ottieni il titolo del canto
    terzine = []

    for lg in c.xpath('.//lg'):
        # Ogni verso viene unito in una stringa con uno spazio come separatore
        terzina = [l.text.strip() for l in lg.xpath('./l') if l.text.strip()]
        terzine.append(terzina)

    canti[h_canto[0]] = terzine

# Genera il risultato finale
data = []
for titolo, ter in canti.items():
    i = 1
    for t in ter:
        # Controlla il numero di versi nella terzina
        if len(t) == 3:  # È una terzina
            range_versi = f"{i}_{i+2}"  # Range per una terzina
            frase = " ".join(t)  # Combina i versi in una stringa unica
            data.append([cantica, titolo, range_versi, frase])
            i += 3
        else:  # Singolo verso o gruppo di versi non standard
            for verso in t:
                range_versi = f"{i}"  # Singolo verso
                data.append([cantica, titolo, range_versi, verso])
                i += 1

df = pd.DataFrame(data, columns=["cantica", "canto", "range_versi", "terzina"])
df.to_csv("inferno_terzine.csv", index=False, sep=';', encoding='utf-8-sig')
print(df)

      cantica  canto range_versi  \
0     Inferno      I         1_3   
1     Inferno      I         4_6   
2     Inferno      I         7_9   
3     Inferno      I       10_12   
4     Inferno      I       13_15   
...       ...    ...         ...   
1591  Inferno  XXXIV     127_129   
1592  Inferno  XXXIV     130_132   
1593  Inferno  XXXIV     133_135   
1594  Inferno  XXXIV     136_138   
1595  Inferno  XXXIV         139   

                                                terzina  
0     Nel mezzo del cammin di nostra vita mi ritrova...  
1     Ahi quanto a dir qual era è cosa dura esta sel...  
2     Tant'è amara che poco è più morte; ma per trat...  
3     Io non so ben ridir com'i' v'intrai, tant'era ...  
4     Ma poi ch'i' fui al piè d'un colle giunto, là ...  
...                                                 ...  
1591  Luogo è là giù da Belzebù remoto tanto quanto ...  
1592  d'un ruscelletto che quivi discende per la buc...  
1593  Lo duca e io per quel cammino ascoso in